In [1]:
# coding: utf-8
import math
import os
import pandas as pd
import numpy as np

# 讀檔
Dot_txt = '.txt'
query_path = 'C:/Users/dnc22/IR_HW1/queries/'
doc_path = 'C:/Users/dnc22/IR_HW1/docs/'

query_list_file = open('query_list' + Dot_txt,'r')
doc_list_file = open('doc_list' + Dot_txt,'r')

query_idf = dict()
Query_tfidf = dict()

for QueryFile_Name in query_list_file.readlines():
    #讀入所有 Query Words 
    QueryFile_Name = QueryFile_Name.strip('\n')
    
    query_file = open(query_path + QueryFile_Name + Dot_txt ,'r')
    query = query_file.readlines()
    query_file.close()
    
    query_dict = dict()
    #截掉中間空白
    query = [[word.lower() for word in doc.split()] for doc in query]
    
    query_tf = dict()
    
    idf = {}
    for words in query[0]:
        if (words in query_dict):
            query_dict[words] += 1
        else:
            query_dict[words] = 1
            
        #用來計算 IDF，這篇文章出現過這個字就放到list
        if (words not in idf):
            idf[words] = 0
            if(words not in query_idf):
                query_idf[words] = 1
            else:
                query_idf[words] += 1
    QueryCount = len(query_dict)
    for words in query[0]:
        # 算 query 的 TF
        #query_tf[words] = query_dict[words] / QueryCount 
        query_tf[words] = query_dict[words] / 1 

    Query_tfidf[QueryFile_Name] = query_tf
    
query_list_file.close()

QueryFile_Count = len(Query_tfidf)

# 計算 word 的 IDF  
for sentense in query_idf:
    query_idf[sentense] = math.log10(1+((QueryFile_Count+1) / (query_idf[sentense]+1)))
    
# 計算 word 的 TF-IDF
for queryFile in Query_tfidf:
    for sentense in Query_tfidf[queryFile]:
        Query_tfidf[queryFile][sentense] = Query_tfidf[queryFile][sentense] * query_idf[sentense]
        
    #做正規化    
    value_list = []
    for key, values in Query_tfidf[queryFile].items():
        value_list.append(values)
    result_score = np.linalg.norm(value_list)
    value_list /= result_score

    #把 list的數字轉回 dict
    index = 0
    for sentense in Query_tfidf[queryFile]:
        Query_tfidf[queryFile][sentense] = value_list[index]
        index += 1
        
#print(Query_tfidf)        
#-------------------

Document_idf = dict()
Document_tfidf = dict()

#讀所有的 Document 檔名，再逐一打開每篇 Document
for Document_Names in doc_list_file.readlines():
    Document_Names = Document_Names.strip('\n')
    
    document_tf = dict()
    #打開 當篇 Docunment
    doc_file = open(doc_path + Document_Names + Dot_txt, 'r')
    document = doc_file.readlines()
    doc_file.close()

    document = [[word.lower() for word in doc.split()] 
               for doc in document]
    
    document_dict = dict()
    idf = []

    for words in document[0]:
        if (words in document_dict):
            document_dict[words] += 1
        else:
            document_dict[words] = 1 
            
        if (words not in idf):
            if(words not in Document_idf):
                Document_idf[words] = 1
                idf.append(words)
            else:
                Document_idf[words] += 1
                idf.append(words)
    DocumentLength = len(document[0])
    # 算 Document內的 Word 的 TF
    for words in document[0]:
        #document_tf[words] = document_dict[words] / DocumentLength
        document_tf[words] = document_dict[words] / 1
    Document_tfidf[Document_Names] = document_tf
    
DocuFile_Count = len(Document_tfidf)
doc_list_file.close()

#計算 Document 的 IDF
for sentense in Document_idf:
    Document_idf[sentense] = math.log(DocuFile_Count / Document_idf[sentense])
    
# 計算 Document 的 TF-IDF
for docFile in Document_tfidf:
    for sentense in Document_tfidf[docFile]:
        Document_tfidf[docFile][sentense] = math.log10(Document_tfidf[docFile][sentense]) * Document_idf[sentense]
        
    #做正規化    
    value_list = []
    for key, values in Document_tfidf[docFile].items():
        value_list.append(values)
    result_score = np.linalg.norm(value_list)
    value_list /= result_score

    #把 list的數字轉回 dict
    index = 0
    for sentense in Document_tfidf[docFile]:
        Document_tfidf[docFile][sentense] = value_list[index]
        index += 1
        
#print(Document_tfidf)        
#-------------------

#算 Cos sim並寫檔
qanswer = dict()
#print(Document_idf)

for queryfile in Query_tfidf:
    qword_list = []
    qvalue_list = []

    #print(Query_tfidf)
    for key, values in Query_tfidf[queryfile].items():
        qword_list.append(key)
        
#         if(key in Document_idf):
#             Query_tfidf[queryfile][key] = Query_tfidf[queryfile][key] * Document_idf[key]
#         else:
#             Query_tfidf[queryfile][key] = 0
        qvalue_list.append(values)
        
        print(Query_tfidf[queryfile])

    # queryfile = '301'
    # qword_list = ['intern', 'organ', 'crime']
    # qvalue_list = [0.333333, 0.3333333, 0.333333]
    # print(qvalue_list)
    
    
    # 湊出文件的向量
    # docfile = FBIS3-10082
    # dvalue_list = [0, 0, 0.039688973194635234]
    danswerlist = []
    for docfile in Document_tfidf:
        dvalue_list = []
        danswer = dict()
        for word in qword_list:
            if(word in Document_tfidf[docfile]):
                dvalue_list.append(Document_tfidf[docfile][word])
            else:
                dvalue_list.append(0)
        
        # 兩向量做 Cos-sim
        #print(qvalue_list, dvalue_list)
        dot = sum(a * b for a, b in zip(qvalue_list, dvalue_list))
        norm_a = sum(a * a for a in qvalue_list) ** 0.5
        norm_b = sum(b * b for b in dvalue_list) ** 0.5
        
        # Cosine similarity
        if(dot == 0 or (norm_a * norm_b) == 0):
            cos_sim = 0
        else:
            cos_sim = dot / (norm_a * norm_b)
        danswer[docfile] = cos_sim
        danswerlist.append(danswer)
    #print(danswerlist)
    qanswer[queryfile] = danswerlist
#print(qanswer)

# 輸出檔案
outPath = 'C:/Users/dnc22/IR_HW1/output.txt'
with open(outPath, 'w') as f:
    f.writelines('Query,RetrievedDocuments\n')
    
    for queryfile in Query_tfidf:
        answerdict = dict()
        N = 0
        while N < len(qanswer[queryfile]):
            answerdict[str(list(qanswer[queryfile][N].keys())[0])] = list(qanswer[queryfile][N].values())[0]
            N = N + 1
        answerdict = sorted(answerdict.items(), key=lambda x:x[1], reverse = True)
        #print(answerdict)
        f.writelines(queryfile + ",")
        for DocuFile in answerdict:
            f.writelines(DocuFile[0] + " ")
        f.writelines("\n")


<ipython-input-1-cd9e52abdae4>:139: RuntimeWarning: invalid value encountered in true_divide
  value_list /= result_score


{'intern': 0.5672438812072306, 'organ': 0.6431492047837458, 'crime': 0.5143865080063751}
{'intern': 0.5672438812072306, 'organ': 0.6431492047837458, 'crime': 0.5143865080063751}
{'intern': 0.5672438812072306, 'organ': 0.6431492047837458, 'crime': 0.5143865080063751}
{'poliomyel': 0.5312340740155382, 'and': 0.39162619398421356, 'post': 0.5312340740155382, 'polio': 0.5312340740155382}
{'poliomyel': 0.5312340740155382, 'and': 0.39162619398421356, 'post': 0.5312340740155382, 'polio': 0.5312340740155382}
{'poliomyel': 0.5312340740155382, 'and': 0.39162619398421356, 'post': 0.5312340740155382, 'polio': 0.5312340740155382}
{'poliomyel': 0.5312340740155382, 'and': 0.39162619398421356, 'post': 0.5312340740155382, 'polio': 0.5312340740155382}
{'hubbl': 0.5773502691896257, 'telescop': 0.5773502691896257, 'achiev': 0.5773502691896257}
{'hubbl': 0.5773502691896257, 'telescop': 0.5773502691896257, 'achiev': 0.5773502691896257}
{'hubbl': 0.5773502691896257, 'telescop': 0.5773502691896257, 'achiev': 0

{'adopt': 0.5773502691896257, 'biolog': 0.5773502691896257, 'parent': 0.5773502691896257}
{'adopt': 0.5773502691896257, 'biolog': 0.5773502691896257, 'parent': 0.5773502691896257}
{'adopt': 0.5773502691896257, 'biolog': 0.5773502691896257, 'parent': 0.5773502691896257}
{'black': 0.5773502691896257, 'bear': 0.5773502691896257, 'attack': 0.5773502691896257}
{'black': 0.5773502691896257, 'bear': 0.5773502691896257, 'attack': 0.5773502691896257}
{'black': 0.5773502691896257, 'bear': 0.5773502691896257, 'attack': 0.5773502691896257}
{'viral': 0.7071067811865475, 'hepat': 0.7071067811865475}
{'viral': 0.7071067811865475, 'hepat': 0.7071067811865475}
{'risk': 0.5999882716312075, 'of': 0.5291768587249377, 'aspirin': 0.5999882716312075}
{'risk': 0.5999882716312075, 'of': 0.5291768587249377, 'aspirin': 0.5999882716312075}
{'risk': 0.5999882716312075, 'of': 0.5291768587249377, 'aspirin': 0.5999882716312075}
{'alzheim': 0.5, 's': 0.5, 'drug': 0.5, 'treatment': 0.5}
{'alzheim': 0.5, 's': 0.5, 'drug